<a href="https://colab.research.google.com/github/burakerdal66/Makine_Ogrenmesi_Projeleri/blob/main/sahteHaberTespiti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**İŞ AKIŞI:**


1.   Haber Verilerinin alınması
2.   Verilerin Ön işlenmesi(PreProcessing)
3. Verileri eğitim ve test olarak bölme
4. Önceden işlenmiş verileri *lojistik regresyon* ile sınıflandırma yapacağız.
5. Eğitim verilerini kullandıktan sonra bu test verilerinde doğruluk değerini buluyoruz. Doğruluk değerine göre de gerçek mi sahte mi olduğunu anlıyoruz.


```
Şimdi Kodlamaya Geçelim
1: Sahte Haber 0: Gerçek Habere
```




In [ ]:
#Bağımlılıkların içe aktarılması: İmporting the Dependencies
import numpy as np
import pandas as pd
import re #Düzenli ifadeler (Regular experions kullanmamızı sağlar)
from nltk.corpus import stopwords #Doğal Dil araç kitini ekleme.
from nltk.stem.porter import PorterStemmer # bir kelimeyi alır ve onun ön-son ekini kaldırır.
from sklearn.feature_extraction.text import TfidfVectorizer # metinsel verilerin sayısal verilere çevrilmesinde rol oynar.
from sklearn.model_selection import train_test_split # verileri eğitim ve test olarak bölmemizi sağlar.
from sklearn.linear_model import LogisticRegression # Makine öğrenmesi algoritmalarından lojistik regresyonu çağırır.
from sklearn.metrics import accuracy_score # Doğruluk değerini bulmak için kullanırız.


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#İngilizce için engellenecek kelimeleri yazdırma.
#Printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data preprocessing

Veri Önişleme


In [ ]:
# Veri setini bir pandas veri çerçevesine olarak yüklemek:
# Loading to a pandas dataframe of dataset
news_dataset=pd.read_csv('/content/train.csv') #Veri setinin yüklenmesi

In [ ]:
news_dataset.shape #Veri setinin boyutunu gösterir

(20800, 5)

In [ ]:
# veriçerçevesinin ilk 5 satırını yazdırma
#print the first 5 rows of the dataframe
news_dataset.head()
#Label olan etiketteki 1 ler sahte haber, 0 lar gerçek haberlerdir.

In [ ]:
#veri setindeki eksik değerlerin sayısını saydırma
#counting the number of missing values in dataset
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

EĞER KÜÇÜK BİR VERİ SETİ VARSA YUKARIDAKİ KAYIP DEĞERLERİNİ SİLME İŞLEMİNİ YAPMAK DOĞRU OLMAZ ÇÜNKÜ VERİLER AZ OLDUĞUNDAN İYİ TAHMİNLEME YAPILAMAZ.

In [ ]:
# boş değerleri boş dize ile değiştirmek
#replacing the null values with empty string
news_dataset= news_dataset.fillna('')

In [ ]:
# Başlık sütunun ile yazar sütununu birleştirme.
# merging the title columnn and author column
news_dataset['content']= news_dataset['author']+' -> '+news_dataset['title']

In [ ]:
print(news_dataset['content'])

0        Darrell Lucus -> House Dem Aide: We Didn’t Eve...
1        Daniel J. Flynn -> FLYNN: Hillary Clinton, Big...
2        Consortiumnews.com -> Why the Truth Might Get ...
3        Jessica Purkiss -> 15 Civilians Killed In Sing...
4        Howard Portnoy -> Iranian woman jailed for fic...
                               ...                        
20795    Jerome Hudson -> Rapper T.I.: Trump a ’Poster ...
20796    Benjamin Hoffman -> N.F.L. Playoffs: Schedule,...
20797    Michael J. de la Merced and Rachel Abrams -> M...
20798    Alex Ansary -> NATO, Russia To Hold Parallel E...
20799           David Swanson -> What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [ ]:
# verileri ve etiketleri(Çıktıları) ayrmak
# seperate the data and label
X= news_dataset.drop(columns='label', axis=1) #sütunlardan "label" ismini kaldırır.
Y= news_dataset['label']

In [ ]:
print(X,"--------")
print(Y)

Stemming:
Stemming, Bir kelimeyi kök kelimesine indirgeme işlemidir. Kelimedeki Önek ve sonek leri kaldırır.

örnek: actor,actress,acting --> act

In [ ]:
port_stem=PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content
  """
  def ile tanımlanan fonksiyona stemmer(kök oluşturma) anlamında bir işlev verdik
  daha sonrar girdi değerlerimizi content(içerik) olarak verdik. sonra düzenli ifadeler kütüphanesi çağrıldı (sub). alfabede olmayan değerler kaldırıldı.
  alfabe ve kelimeler dışındakiler kaldırıldıktan sonra " " ile bunların yerini boşluk işe doldur.
  daha sonrar lower ile tüm kelimeleri küçük harfe çevir.
  split ile hepsi bölünerek küçük listelere dönüştürülüyor.
  daha sonra kök çıkarma fonksiyonunu çağırıyoruz.  önemsiz kelimeleri(stopwords) kaldırıyoruz.
  daha sonra tüm kelimeleri birleştiriyoruz.
  en sonunda işlenmiş veriyi döndürüyoruz.
   """

In [ ]:
news_dataset["content"] = news_dataset["content"].apply(stemming)

In [ ]:
print(news_dataset["content"])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [ ]:
# data ve labelleri ayırmak
# seperate the data and label
X= news_dataset['content'].values
Y= news_dataset['label'].values # sonuç değerleri

In [ ]:
print(X)
print(Y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']
[1 0 1 ... 0 1 1]


In [ ]:
Y.shape

(20800,)

In [ ]:
# metinsel verileri sayısal verilere dönüştürme.
# converting the textual data to numerical data
vectorizer=TfidfVectorizer()
vectorizer.fit(X)
X=vectorizer.transform(X)
#

In [ ]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

Dataseti eğitim ve test verileri olarak bölmek:

Splitting the dataset to train and test data

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

Lojistik regresyon ile Model Eğitimi

Training model with logistic regression

In [ ]:
model=LogisticRegression()

In [ ]:
model.fit(X_train,Y_train)

LogisticRegression()

Değerlendirme Aşaması

Evalution

In [ ]:
# Eğitim verisi üzerinde doğruluk değeri
# Accuracy score on the training data
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,Y_train)
# modelimizin ekstrem veriler üzerinde ve bu modeller üzerindeki etiketler üzerinde tahmin yapmasını istiyoruz.


In [ ]:
print("Eğitim verisinin doğruluk değeri:  ",training_data_accuracy)
print("accuracy score of training data: ", training_data_accuracy)

Eğitim verisinin doğruluk değeri:   0.9865985576923076
accuracy score of training data:  0.9865985576923076


Örneğin birisi bir sınava çalışıyor. sınava çalışırken test kitaplarındaki soruları çözer ve sınavda çıkabilecek sorulara göre kendisini test eder. bu eğitim verilerinin doğruluğu olmuş oldu. Şimdi de sınava girdikten sonra alacağı sonuçları test etmek için test verilerini eğiteceğiz.

In [ ]:
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(X_test_prediction,Y_test)

In [ ]:
print("Test verisinin doğruluk değeri:  ",test_data_accuracy)
print("accuracy score of test data: ", test_data_accuracy)

Test verisinin doğruluk değeri:   0.9790865384615385
accuracy score of test data:  0.9790865384615385


Modelin eğitim verileri test verielrine çok yakın olduğu için aşırı öğrenme(overfitting) yapmıyor.

Şimdi de bir tahminleme sistemi yapacağız.

We will making to prediction system

In [ ]:
X_new= X_test[1]
prediction= model.predict(X_new)
print(prediction)
if (prediction[0]==0):
  print("This news is Real")
else:
  print("This news is Fake")


[0]
This news is Real


In [ ]:
if (Y_test[0]==1):
  print("This news is Real")
else:
  print("This news is Fake")

This news is Real


Burada gidiğimiz haber test edilen X verilerinin ilk karekteri oldu. Burada X_new değeri tahmin değerine atıldı ve koşul içinde gerçek mi sahte mi olduğu sorgulandı  ve sahte denildi

print(Y_test[0]) ile de baktığımızda haber sahte olduğu söylenmişti. bu sebeple tahminleme sistemimiz doğru çalıştığını gördük.